In [30]:
%matplotlib notebook
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import datetime as dt
import pandas as pd


In [31]:
from bs4 import BeautifulSoup

In [32]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline, FeatureUnion
import pymorphy2 # Морфологический анализатор.
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier

In [33]:
from scipy.stats import randint as sp_randint
from time import time
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK, rand

In [34]:
# отключаем warnings
import warnings

warnings.filterwarnings('ignore')

In [35]:
# cleantext = BeautifulSoup(raw_html, "lxml").text

In [36]:
df_train_raw = pd.read_csv('data/train.csv', sep='\t')

In [37]:
df_train_raw.head()

,id,name,description,target
0,0,Заведующий отделом/секцией в магазин YORK (Уру...,<p><strong>В НОВЫЙ МАГАЗИН YORK (хозтовары) пр...,1
1,1,Наладчик станков и манипуляторов с ПУ,Обязанности:работа на токарных станках с ЧПУ T...,0
2,2,Разработчик С++ (Криптограф),<strong>Требования:</strong> <ul> <li>Опыт про...,0
3,3,Фрезеровщик,<p>Условия:</p> <ul> <li>На работу вахтовым ме...,0
4,4,Мерчендайзер/продавец-консультант,<p><strong>Компания Палладиум Стандарт - призн...,1


In [38]:
df_train_raw.shape

(200000, 4)

In [39]:
df_test = pd.read_csv('data/test.csv', sep='\t')

In [40]:
df_test.head()

,id,name,description
0,200000,Дизайнер-консультант мебели,<p><strong>Обязанности:</strong></p> <ul> <li>...
1,200001,Продавец-консультант (ТЦ на Пушкина),<p><strong>Обязанности</strong>:</p> <p>∙ конс...
2,200002,Менеджер по продажам,<p>Торговый Дом «Форт» это ведущая компания Пе...
3,200003,Продавец-консультант в магазин одежды (ТЦ Волн...,<p><strong>Требуются продавцы консультанты в м...
4,200004,Специалист по охране труда,<strong>Обязанности:</strong> <ul> <li> <p>осу...


In [41]:
import re

def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext
    

In [42]:
def prepare_data(dataframe):
    df = dataframe.copy()
    # clearhtml
    df['description'] = df['description'].apply(lambda x : cleanhtml(x) )
    # replace &quot to "
    df['description'] = df['description'].str.replace('&quot;','"')
    df['description'] = df['description'].str.replace('\u200b','')
    # uni name and description
    df['text'] = df['name'] + ' ' + df['description'] 
    # drop
    print(type(df))
    df = df.drop(['name', 'description', 'id'], axis=1)
    
    return df
    

In [43]:
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

In [44]:
def f_tokenizer(s):
    morph = pymorphy2.MorphAnalyzer()
    if type(s) == str:
        t = s.split(' ')
    else:
        t = s
    f = []
    for j in t:
        line = re.sub(r"[\s\d\+\:\;\(\)\|\/\%\,\-\.\$\#\@\!\*]", "", j)
        m = morph.parse(line)
        if len(m) != 0 :
            if line is not (''):
                wrd = m[0]
                if wrd.tag.POS not in ('NUMR','PREP','CONJ','PRCL','INTJ'):
                    tag = str(wrd.tag.POS) if wrd.tag.POS is not None else "None"
                    f.append( "{word}_{tag}".format(word=wrd.normal_form, tag=tag))
    return f

In [45]:
%%time 
# N = df_train_raw.shape[0]
N = 1000
df_train = prepare_data(df_train_raw[:N])

<class 'pandas.core.frame.DataFrame'>
CPU times: user 45.2 ms, sys: 11.6 ms, total: 56.8 ms
Wall time: 55.5 ms


In [46]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 2 columns):
target    1000 non-null int64
text      1000 non-null object
dtypes: int64(1), object(1)
memory usage: 15.7+ KB


In [47]:
df_tmp_test = prepare_data(df_test[:N])


<class 'pandas.core.frame.DataFrame'>


In [48]:
df_tmp_test.head()

,text
0,Дизайнер-консультант мебели Обязанности: Рабо...
1,Продавец-консультант (ТЦ на Пушкина) Обязаннос...
2,Менеджер по продажам Торговый Дом «Форт» это в...
3,Продавец-консультант в магазин одежды (ТЦ Волн...
4,Специалист по охране труда Обязанности: осущ...


In [49]:
df_id = df_test[:].drop(['name', 'description'], axis=1)

In [50]:
df_id.head()

,id
0,200000
1,200001
2,200002
3,200003
4,200004


In [51]:
label = 'target'
RND_SEED = 123
idx_features = df_train.columns != label

In [52]:
X = df_train.loc[:, idx_features].values.tolist()
y = df_train.loc[:, ~idx_features].values
tests = df_tmp_test["text"].values

In [53]:
texts = []
for item in X:
    texts.append(item[0])

In [54]:
texts[10]

'Backend Developer (Node.js/NoSQL) Сегодня Плэйкот - это 8 лет истории, 110 профессионалов и 3 игры в Top Grossing Facebook. Наш подход к работе прост: мы даем талантливым, увлеченным играми и ценящим качество людям возможность принимать решения самостоятельно. Многим трудно поверить, но у нас слово команды в том, что и как делать в игре, действительно является решающим.   Серверная часть эпического проекта "Age of Magic" ждет своего героя! Совсем скоро состоится глобальный запуск игры, и к этому моменту все должно быть идеально. И, конечно же, стабильно!   Что мы вам доверим:  разработку и поддержку сервера игры; продумывание и разработку сервисов и инструментов для мониторинга и технической поддержки игры.    Наши пожелания к кандидату:  опыт разработки серверной части приложения от 2-х лет; отличное знание JavaScript; опыт работы с высоконагруженными проектами; опыт работы с PostgreSQL; опыт работы с NoSQL (MongoDB/DynamoDB).    Также придутся кстати:  опыт работы с Amazon Web Servi

In [55]:
def run_trials_template(X, y, params, evals=100):

    def hyperopt_cv(X, y, params):
        
        X_ = X.copy()
        
        # Отделяем параметры лог регрессии в отдельный словарь
        lm_params = {}
        for k, v in params.items():
            if k.startswith('glob'):
                continue                
            elif k.startswith('clf'):
                clf_params[k.split('_', 1)[1]] = v
            elif k.startswith('tfid'):
                tfid_params[k.split('_', 1)[1]] = v
        
        #
        tdidf = TfidfVectorizer(**tfid_params)
        # Создаем лог рег с нужными параметрами
        clf = RandomForestClassifier(**clf_params)
        
        # Итоговый пайплайн
        model = Pipeline([
            ('tdidf', tdidf),
            ('clf', clf)
        ])

        # Схема кросс-валидации
        n_splits = 5
        cv = StratifiedKFold(n_splits=n_splits, shuffle=True, 
                             random_state=RND_SEED)
        
        scores = cross_val_score(model, X_, y,
                                 scoring='roc_auc', 
                                 cv=cv, 
                                 n_jobs=-1)

        # Возвращаем среднее значение метрики и отклонение (на всякий случай)
        return scores.mean(), scores.std()

    def f(params):
        acc, std = hyperopt_cv(X, y, params)
        return {'loss': -acc, 'qscore': -acc, 'qscore_std': std, 'status': STATUS_OK}

    trials = Trials()
    best = fmin(f, 
                params, 
                algo=tpe.suggest, 
                max_evals=evals, 
                trials=trials, 
                verbose=1)
    
    return trials

In [56]:
# model RandomForestClassifier
# http://scikit-learn.org/stable/auto_examples/model_selection/\
# plot_randomized_search.html#sphx-glr-auto-examples-model-selection-plot-randomized-search-py

#https://www.kaggle.com/nikitabu/tfidf-lightgbm-pipeline-with-randomsearchcv/code
#

# clf = RandomForestClassifier()

# model_3 = Pipeline([
#         ('union', FeatureUnion(
#                 # Use FeatureUnion to combine the features from subject and body
#             transformer_list=[
#                     # Pipeline for standard bag-of-words model
#                 ('text', Pipeline([
#                         ('tfidf', TfidfVectorizer(
#                             min_df=15,
#                             max_df=.95,
#                             tokenizer=f_tokenizer,
#                             norm=None, 
#                             smooth_idf=False,
#                         )),
#                     ])
#                 ),
#             ],# end  transformer_list
#                 # weight components in FeatureUnion
#             transformer_weights={
#                 'text': 1.0,
#             }) # end FeatureUnion
#         ),# end union
#         ('clf', clf),
# ]) # end pipeline
n_iter_search = 1000
n_folds = 5 # Define number of CV folds

In [57]:
param_dist = {
            "tdidf__min_df" : [1,5,10,15],
            "tdidf__max_df": np.arange(0.8, 1., 0.02),
            "tdidf__tokenizer" : f_tokenizer,
            "tdidf__norm": None,
            "tdidf__smooth_idf": [True, False],
            "clf__max_depth": [10,20,30,40, None],
            "clf__max_features": "auto",
            "clf__n_estimators": np.arange(0, 1100, 100),
            "clf__min_samples_split": sp_randint(1, 131),
            "clf__min_samples_leaf": sp_randint(1, 131),
            "clf__bootstrap": [True, False],
            "clf__criterion": ["gini", "entropy"],
            }


In [58]:
trials = run_trials_template(X, y, param_dist, evals=40)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [29]:
predictions = random_search.predict(tests)
df_id['target'] = pd.DataFrame(predictions.tolist())

In [30]:
df_id.shape

(170179, 2)

In [31]:
from datetime import datetime
today = datetime.today().strftime('%Y-%m-%d_%H-%M')
df_id.to_csv(path_or_buf='./submission/submission_{today}.cvs'.format(today=today), index=False, sep=',')

In [32]:
print("<<< DONE!!! >>>")

<<< DONE!!! >>>
